## Personas of Thought
Asking the LLM to generate a crowd of personas to answer a question before aggregating their feedback.

### Imports
Load helper functions for sending prompts (`generate_text`) and rendering outputs (`render_markdown`).

In [1]:
from src.openai_client import generate_text
from src.fnUtils import render_markdown

### Model Settings
Set the model, temperature, and other defaults used across all prompts.

In [2]:
# Configuration for generate_text calls
model_name = "gpt-4.1-nano"
temperature = 0.7

### Baseline (Naive) Response
Ask the core question once to capture an unconstrained single-paragraph answer for comparison.

In [3]:
question = "What distinguishes successful startup companies?"
number = 10

system_prompt = "You are a helpful assistant."
question_prompt_template = "Answer this question: {question}. Please respond in a single paragraph. Limit your response to approximately 200 words."

question_prompt = question_prompt_template.format(question=question)
naive_response = generate_text(prompt=question_prompt, 
                                model_name=model_name, 
                                system_instruction=system_prompt, 
                                temperature=temperature)

render_markdown("### Naive Response\n" + naive_response)


> ### Naive Response
> Successful startup companies are distinguished by a combination of innovative value propositions, adaptability, strong leadership, and a deep understanding of their target market. They typically identify a clear problem or unmet need and develop a unique solution that differentiates them from competitors. Flexibility and agility are crucial, allowing startups to pivot quickly in response to market feedback and changing conditions. Effective resource management, including securing adequate funding and building a dedicated, skilled team, also plays a vital role. Successful startups foster a culture of resilience, continuous learning, and customer-centricity, ensuring they remain aligned with user needs and preferences. Additionally, they leverage technology and data-driven decision-making to optimize operations and scale efficiently. Strong networking and strategic partnerships can accelerate growth, while a compelling vision and passion motivate teams and attract investors. Ultimately, the most successful startups combine innovation with pragmatic execution, persistent problem-solving, and an unwavering focus on delivering value to their customers, enabling them to grow sustainably and establish a competitive presence in their industry.

### Expert Ensemble Prompt
Generate multiple expert viewpoints, then synthesize them into a joint statement. Demonstrates role-specific prompting and structured outputs.

In [7]:
experts_prompt_template = """Give me a single paragraph response to the following question: {question}

# Instructions:
1. Name {number} world-class experts (past or present) who would be great at answering this question. Select experts who have directly addressed or studied topics related to the question, based on their published work and professional experience.
2. For each expert, please answer the question critically from their perspective given their background and experience. Avoid relying on stereotypes or generalizations about the experts. Base your responses on specific examples of their work and contributions. Limit each expert response to approximately 100 words.
3. **Synthesize the individual expert responses into a single, unified answer to the original question.  Imagine the experts are collaborating to write a joint statement.  Identify the strongest arguments, address any conflicting viewpoints by proposing a reconciled perspective, and create a cohesive, insightful, and direct answer to the question. The combined response should not simply summarize the individual opinions but rather build upon them to form a more complete and well-supported conclusion. This is the definitive answer to the question, informed by all the experts' perspectives. Limit the final response to approximately 200 words.**

# Output format:
Here's an example of how the output should be structured:
## Expert names:
* **Expert name** (relevant experience)
* **Expert name** (relevant experience)
* **Expert name** (relevant experience)
...

## Expert responses:
* **Expert name**: <response>
* **Expert name**: <response>
* **Expert name**: <response>
...

## Final response:  This is the combined response, synthesizing the expert perspectives.

Note that both individual expert responses and the final combined response should be written anonymously, without explicitly attributing ideas to specific individuals beyond the initial expert list.
"""
experts_prompt = experts_prompt_template.format(question=question, number=number)
# print(experts_prompt)
experts_response = generate_text(prompt=experts_prompt, model_name=model_name, system_instruction=system_prompt, temperature=temperature)

render_markdown("### Experts Response\n" + experts_response)


> ### Experts Response
> ## Expert names:
> * Peter Thiel (co-founder of PayPal, author of "Zero to One")
> * Eric Ries (author of "The Lean Startup")
> * Steve Blank (entrepreneur and educator, creator of the Customer Development methodology)
> * Reid Hoffman (co-founder of LinkedIn, venture capitalist)
> * Saras Sarasvathy (professor and researcher on Effectuation theory)
> * Jim Collins (author of "Good to Great")
> * Ben Horowitz (co-founder of Andreessen Horowitz, author of "The Hard Thing About Hard Things")
> * Clayton Christensen (professor and author of "The Innovator’s Dilemma")
> * Whitney Johnson (expert on disruptive innovation and personal growth)
> * Ash Maurya (author of "Running Lean" and lean startup advocate)
> 
> ## Expert responses:
> * **Peter Thiel**: Successful startups often focus on creating monopoly through innovative products or services that drastically differentiate themselves, rather than competing in crowded markets. The emphasis is on breakthrough ideas that can scale rapidly and establish a dominant position early on.
> * **Eric Ries**: Success hinges on validated learning through rapid experimentation and customer feedback. Startups must be agile, iteratively testing hypotheses, and pivoting when necessary to find a sustainable business model.
> * **Steve Blank**: The key is understanding customer needs deeply before scaling. Successful startups develop a repeatable, scalable business model through a disciplined process of customer discovery and validation.
> * **Reid Hoffman**: Building a strong, adaptable team and network is crucial, along with maintaining a resilient mindset. Success also depends on timing and the ability to leverage network effects early.
> * **Saras Sarasvathy**: Effectuation principles—starting with available means, forming partnerships, and co-creating opportunities—are vital, as they enable startups to navigate uncertainty and co-evolve with their environment.
> * **Jim Collins**: Enduring success results from disciplined people, a clear vision, and a culture of continuous improvement, rather than just innovative ideas or rapid growth.
> * **Ben Horowitz**: Overcoming the "hard things"—such as managing crises, maintaining culture, and making tough decisions—is what separates successful startups. Resilience and leadership are key.
> * **Clayton Christensen**: Disruptive innovations can create new markets and opportunities for startups to succeed by initially targeting overlooked segments and eventually reshaping industries.
> * **Whitney Johnson**: Personal and organizational growth driven by embracing change, taking risks, and continuously challenging the status quo are essential for long-term startup success.
> * **Ash Maurya**: Lean methodologies, customer-centric product development, and disciplined iteration help startups avoid wasting resources and increase their chances of finding a product-market fit.
> 
> ## Final response:
> Successful startup companies distinguish themselves through a combination of innovation, agility, customer focus, and resilience. They often create unique value propositions that disrupt existing markets or establish new ones, leveraging principles like validated learning and rapid experimentation. A strong, adaptable team and network are vital, as is a disciplined approach to discovering and validating a scalable business model. Navigating uncertainty with effectuation, maintaining a culture of continuous improvement, and overcoming inevitable crises are crucial. Ultimately, success depends on balancing innovative vision with disciplined execution, fostering growth through customer insights, strategic pivots, and resilient leadership to sustain long-term value creation.

### Persona Ensemble Prompt
Elicit diverse demographic perspectives and merge them into a single inclusive answer. Shows how to guide style, format, and synthesis rules.

In [9]:
personas_prompt_template = """Give me a single paragraph response to the following question: {question}

# Instructions:
1. Name {number} distinct demographic personas who would be relevant for answering this question. Select personas representing diverse backgrounds, experiences, and perspectives relevant to the question.  Focus on specific demographics that significantly influence their point of view (e.g., age, income, location, education, occupation, family status).
2. For each persona, answer the question critically from their perspective, deeply considering their specific background, life experiences, values, and priorities.  Imagine you are fully embodying this persona. Be specific about how their demographic characteristics shape their answer. Limit each persona's response to approximately 100 words.
3. **Synthesize the individual persona responses into a single, unified answer to the original question. Imagine these personas are collaborating in a focus group or town hall to reach a consensus.  Identify common ground and key differences in their viewpoints. Address any conflicting perspectives by proposing a nuanced and inclusive solution that acknowledges and integrates the needs and concerns of each persona. The combined response should not simply summarize the individual opinions but rather build upon them to form a more complete, empathetic, and well-reasoned answer to the question. This is the definitive answer to the question, informed by all the personas' diverse experiences and perspectives. Limit the final response to approximately 200 words.**

# Output format:
## Persona names:
* **Persona name** (relevant demographics: e.g., 35-year-old, urban, single professional)
* **Persona name** (relevant demographics: e.g., 60-year-old, rural, retired teacher)
* **Persona name** (relevant demographics: e.g., 20-year-old, suburban, college student)
...

## Persona responses:
* **Persona name**: <response>
* **Persona name**: <response>
* **Persona name**: <response>
...

## Final response:  <-- **IMPORTANT:  The final synthesized response MUST always begin with the tag "## Final response:"**
This is a single paragraph, synthesizing the expert perspectives.
"""
personas_prompt = personas_prompt_template.format(question=question, number=number)
personas_response = generate_text(prompt=personas_prompt, model_name=model_name, system_instruction=system_prompt, temperature=temperature)

render_markdown(personas_response)


> ## Persona names:
> * **Emma** (30-year-old urban startup founder)
> * **Raj** (45-year-old rural small business owner)
> * **Lina** (22-year-old college student, aspiring entrepreneur)
> * **Michael** (55-year-old corporate executive)
> * **Aisha** (28-year-old social entrepreneur in developing country)
> * **Carlos** (40-year-old immigrant tech worker)
> * **Sophie** (65-year-old retired teacher, mentor to young entrepreneurs)
> * **Jamal** (35-year-old freelancer, gig economy worker)
> * **Naomi** (50-year-old venture capitalist)
> * **Akira** (25-year-old recent graduate, digital nomad)
> 
> ## Persona responses:
> * **Emma**: Successful startups thrive on innovation, agility, and understanding customer needs. As a young founder, I see rapid adaptation and a strong team culture as key drivers of success, along with securing funding and scalable business models.
> * **Raj**: For me, sustainability and community impact matter most. A startup that serves local needs, respects traditions, and creates jobs can succeed by building trust and loyalty, not just by rapid growth.
> * **Lina**: Success involves solving real problems that resonate with users, especially younger generations. Incorporating technology and social consciousness helps startups stand out and attract passionate support.
> * **Michael**: Long-term strategic planning, solid financial fundamentals, and leadership are essential. Success depends on managing risks, maintaining operational excellence, and scaling sustainably.
> * **Aisha**: In developing contexts, success often hinges on addressing basic needs, fostering local entrepreneurship, and ensuring affordability. Social impact and inclusivity are vital for a startup's sustainability.
> * **Carlos**: Innovation and technical expertise are crucial. Success requires leveraging diverse talents, creating solutions that can scale globally, and navigating cultural differences.
> * **Sophie**: Mentorship, resilience, and community support are key. Successful startups often have founders who learn from failures, stay committed, and build networks that sustain growth.
> * **Jamal**: Flexibility and adaptability are my priorities. Success comes from being able to pivot quickly, understand market shifts, and maintain low costs while delivering value.
> * **Naomi**: Investment readiness, strong value propositions, and scalability attract funding. A successful startup demonstrates growth potential and a clear path to profitability.
> * **Akira**: Digital tools and a global outlook help startups succeed. Success involves innovative ideas, agility, and the ability to reach diverse markets quickly.
> 
> ## Final response:
> ## Final response:
> Successful startup companies distinguish themselves through a blend of innovation, adaptability, and understanding of customer needs, as emphasized by entrepreneurs like Emma and Lina. However, sustainability, community impact, and inclusivity—highlighted by Raj and Aisha—are equally vital, ensuring long-term relevance and social good. Strategic planning, strong leadership, and financial discipline, underscored by Michael and Naomi, provide a foundation for growth, while resilience, mentorship, and agility—pointed out by Sophie and Jamal—enable startups to navigate challenges effectively. Leveraging technology, diverse talents, and global perspectives, as noted by Carlos and Akira, broadens opportunities. Ultimately, success arises from a holistic approach that balances innovation, social responsibility, strategic foresight, and adaptability, fostering startups that are not only profitable but also meaningful and sustainable in their communities and beyond.

### Parse Final Answer
Utility functions to extract tagged sections (especially the final synthesized response) from the generated text.

In [10]:
import re

def extract_tag_contents(text, tags):
    """
    Extracts the contents of a tag from a given text.

    Args:
        text (str): The input text.
        tags (tuple): A tuple containing the opening and closing tags.

    Returns:
        str: The extracted contents of the tag, or None if not found.
    """
    open_tag, close_tag = tags
    pattern = rf'{re.escape(open_tag)}(.*?){re.escape(close_tag)}|{re.escape(open_tag)}(.*)'
    match = re.search(pattern, text, re.DOTALL)

    if match:
        return (match.group(1) or match.group(2)).strip()
    else:
        return None

def extract_final_answer(text, tag="## Final response:"):
    if text is None:
        return None
    if "## Combined response:" in text:
        text = text.replace("## Combined response:", "## Final response:")
    pattern = rf'{re.escape(tag)}\s*\n*(.*?)(?=\n\n|$)'
    match = re.search(pattern, text, re.DOTALL | re.IGNORECASE)  # Added re.IGNORECASE for robustness

    if match:
        return match.group(1).strip()
    else:
        return None

def extract_choice(text):
    tags = ("<choice>", "</choice>")
    return extract_tag_contents(text, tags)

print(extract_final_answer(personas_response))

## Final response:
Successful startup companies distinguish themselves through a blend of innovation, adaptability, and understanding of customer needs, as emphasized by entrepreneurs like Emma and Lina. However, sustainability, community impact, and inclusivity—highlighted by Raj and Aisha—are equally vital, ensuring long-term relevance and social good. Strategic planning, strong leadership, and financial discipline, underscored by Michael and Naomi, provide a foundation for growth, while resilience, mentorship, and agility—pointed out by Sophie and Jamal—enable startups to navigate challenges effectively. Leveraging technology, diverse talents, and global perspectives, as noted by Carlos and Akira, broadens opportunities. Ultimately, success arises from a holistic approach that balances innovation, social responsibility, strategic foresight, and adaptability, fostering startups that are not only profitable but also meaningful and sustainable in their communities and beyond.


### Sandbox Cell
Use this space for experiments—tweak prompts, try different questions, or inspect intermediate outputs.